In [5]:
import torch
from torch.utils.data import Dataset
import glob
import os

In [4]:
class NpyDataset(Dataset):
    def __init__(self,samples_path,transforms=None,train_num_samples=None,tokenizer=None):
        if 'coco' in samples_path:
            self.data='coco'
        else:
            self.data='cc3m'
        if os.path.isdir(samples_path):
            # load and merge all splited files in the dir
            data_file_splits=glob.glob(os.path.join(samples_path,'*.npy'))
            print(f'merging {len(data_file_splits)} splied files from {samples_path}')
            self.samples=[]
            for file_split in data_file_splits:
                self.samples.extend(self.loadList(file_split))
        else:
            # load single splited file fiven the path
            self.samples=self.loadList(samples_path)
        if train_num_samples:
            self.samples=self.samples[:train_num_samples]
        self.transforms = transforms
        self.tokenize = tokenizer
    def loadList(self,file_path):
        # the filename should mention the extension '.npy'
        tempNumpyArray=np.load(file_path,allow_pickle=True)
        return tempNumpyArray.tolist()
    def __len__(self):
        return len(self.samples)
    def __getitem__(self,index):
        texts=torch.stack([self.tokenize([str(self.samples[index]['caption'])])[0],
        self.tokenize([str(self.samples[index]['relation_aug_caption'])])[0],
        self.tokenize([str(self.samples[index]['adj_aug_caption'])])[0],
        self.tokenize([str(self.samples[index]['noun_aug_caption'])])[0],
        self.tokenize([str(self.samples[index]['verb_aug_caption'])])[0]])
        if self.data=='coco':
            image_id=self.samples[index]['image_id']
            image_path=os.path.join(COCO_DATASET_ROOT,'train2014',f"COCO_train2014_{'0'*(12-len(str(image_id)))}{image_id}.jpg")
            images = self.transforms(Image.open(image_path).convert("RGB"))
        else:
            images = self.transforms(Image.open(str(self.samples[index]['image_path'])).convert("RGB"))
        valid_caption_mask=torch.tensor(self.samples[index]['valid_caption'])
        return images, texts, valid_caption_mask


In [12]:
import numpy as np
tempNumpyArray=np.load("/home/mila/l/le.zhang/scratch/winonground/data/processed_dataset/coco2014/processed_dataset1.npy",allow_pickle=True)

In [14]:
tempNumpyArray[0],tempNumpyArray[1],tempNumpyArray[2]

({'image_id': 131060,
  'id': 431313,
  'caption': 'A teddy bear perched on a tree branch looking downward.',
  'relation_aug_caption': 'A bear teddy perched on a tree branch looking downward.',
  'adj_aug_caption': '###',
  'noun_aug_caption': 'A teddy Bear perched on a tree branch looking downward.',
  'verb_aug_caption': 'A teddy bear perched on a tree branch looks downward.',
  'valid_caption': [1, 1, 0, 1, 1]},
 {'image_id': 272196,
  'id': 431342,
  'caption': 'A pile of rocks stacked on the top of a toilet seat.',
  'relation_aug_caption': 'A rocks of pile stacked on the top of a toilet seat.',
  'adj_aug_caption': '###',
  'noun_aug_caption': 'A pile of rocks stacked on the top of a bench seat.',
  'verb_aug_caption': 'A pile of rocks perched on the top of a toilet seat.',
  'valid_caption': [1, 1, 0, 1, 1]},
 {'image_id': 521165,
  'id': 431368,
  'caption': 'A small engine plane gearing up for take off.',
  'relation_aug_caption': '###',
  'adj_aug_caption': 'A diesel engine 